# Create Dataset 
this script adds the weather data to the daily summaries 

In [5]:
import pandas as pd
import numpy as np
import datetime

In [6]:
bills = pd.read_csv("./datasets/bills.csv")
city_weather = pd.read_csv("./datasets/cityWeather.csv")
venues = pd.read_csv("./datasets/venues.csv")

In [7]:
# how many order types are we expecting? and what are they? 
order_types = bills['order_take_out_type_label'].unique()
order_types

array(['dinein', 'bartab', 'takeout', 'delivery', 'onlineorder'],
      dtype=object)

In [8]:
# add flag columns 
flagged_data = bills 
for label in order_types: 
    flagged_data[label] = np.where(flagged_data['order_take_out_type_label'] == label, 1, 0)

In [9]:
grouped_dataset = flagged_data.groupby(['venue_xref_id', 'business_date']).agg(
    {'bill_total_billed' : 'sum', 
     'bill_total_discount_item_level' : 'sum',
     'bill_total_gratuity' : 'sum',
     'bill_total_net' : 'sum',
     'bill_total_tax' : 'sum',
     'bill_total_voided' : 'sum',
     'payment_amount' : 'sum', 
     'payment_count' : 'sum', 
     'payment_total_tip' : 'sum', 
     'sales_revenue_with_tax' : 'sum', 
     'order_duration_seconds' : 'mean',
     'dinein' : 'sum', 
     'bartab' : 'sum',
     'takeout' : 'sum', 
     'delivery' : 'sum',
     'onlineorder' : 'sum',
     'waiter_uuid' : 'nunique'
    }).reset_index()


In [10]:
# join venues with daily records
daily = grouped_dataset
venues['city'] = venues['city'].str.lower()
daily_venue = pd.merge(daily, venues, on=["venue_xref_id"], how="inner")

In [11]:
# link the weather to the daily summary and venues dataframe
def getDate(datetime):
    return datetime.split(' ')[0] # get just the date, no time
city_weather['business_date'] = city_weather['date'].apply(getDate)

In [12]:
venue_transaction_weather = pd.merge(daily_venue, city_weather, on=['city', 'country', 'business_date'], how="inner")

In [13]:
venue_transaction_weather.to_csv('./datasets/venue_daily_weather.csv')

In [14]:
holidaylist = pd.read_csv('./datasets/Holidays_CanUS.csv')
mergedweatherwithholidays = pd.merge(venue_transaction_weather, 
                                     holidaylist,  
                                     left_on = ['business_date', 'country'],  
                                     right_on = ['date', 'countryCode'],
                                     how ='left') 

mergedweatherwithholidays = mergedweatherwithholidays.drop(columns=['date_y', 'localName','name','fixed','global','counties','launchYear','types', 'countryCode'])

mergedweatherwithholidays['weekend'] = 0
for index,row in mergedweatherwithholidays.iterrows():
    # adding weekend
    date = datetime.datetime.strptime(row['business_date'], '%Y-%m-%d').date()
    if (date.isoweekday() > 5):
        mergedweatherwithholidays.at[index, 'weekend'] = 1

mergedweatherwithholidays['holiday'] = mergedweatherwithholidays['holiday'].fillna(0)
mergedweatherwithholidays.to_csv('./datasets/WeatherandHolidays.csv', index=False)